In [1]:
import random
import numpy as np
import h5py
import json
import pdb
import io
from PIL import Image
from format import pytree_display

def load_data_from_hdf5(file_list, batch_size, file_batch_size, embedding_dict):
    
  def pad_and_resize(image, target_size):
    original_size = image.size
    ratio = float(target_size) / max(original_size)
    new_size = tuple([int(x * ratio) for x in original_size])
    
    resized_image = image.resize(new_size, Image.Resampling.LANCZOS)
    new_image = Image.new("RGB", (target_size, target_size))
    new_image.paste(resized_image, ((target_size - new_size[0]) // 2, (target_size - new_size[1]) // 2))

    return new_image

  def bytes_image_to_np(image_bytes, image_size=300):
    image = Image.open(io.BytesIO(image_bytes))
    image = pad_and_resize(image, image_size)
    image_array = np.array(image)
    image_array = image_array[:,:,[2,1,0]] / 255.0
    return image_array
  
  file_list = random.sample(file_list, file_batch_size)
  num_per_file = batch_size // file_batch_size
  natural_language_embedding = []
  image = []
  arms_action = []
  
  length = np.random.randint(1, 16, batch_size)
  is_first = np.zeros((batch_size, 15), dtype=bool)
  for i in range(batch_size):
    if length[i] != 15:
      is_first[i, 15 - length[i]] = True
      
  terminate_episode = np.zeros((batch_size, 15, 3), dtype = np.int32)
  terminate_episode[:,:,1] = 1
  
  files = random.sample(file_list, file_batch_size)
  for i, file in enumerate(files):
    with h5py.File(file, 'r') as f:
      natural_language_embedding.extend([embedding_dict[f['instruction'][()].decode('utf-8')] for _ in range(num_per_file)])
      for j in range(num_per_file):
        l = length[i * num_per_file + j]
        traj_l = f['action'].shape[0]
        start = np.random.randint(0, traj_l - l + 1)
        image.append(np.pad(np.array(list(map(bytes_image_to_np, f['observations']['images']['cam_high'][start:start+l]))),
                            ((15 - l, 0), (0, 0), (0, 0), (0, 0)), 'constant'))
        arms_action.append(np.pad(np.array(f['action'][start:start+l]), ((15 - l, 0), (0, 0)), 'constant'))
  
  batch = {
    'is_first': np.array(is_first),
    'observation': {
        'image': np.array(image),
        'natural_language': np.array(natural_language_embedding)
        },
    'action': {
        'arms': np.array(arms_action),
        'terminate_episode': terminate_episode
    },
    'is_last': np.zeros((batch_size, 15), dtype=bool),
    'is_terminal': np.zeros((batch_size, 15), dtype=bool)
    }
  
  return batch

def load_data_from_hdf5_2(file_list, batch_size, file_batch_size, embedding_dict, max_length = 32):
    
    def pad_and_resize(image, target_size):
        original_size = image.size
        ratio = float(target_size) / max(original_size)
        new_size = tuple([int(x * ratio) for x in original_size])
        
        resized_image = image.resize(new_size, Image.Resampling.LANCZOS)
        new_image = Image.new("RGB", (target_size, target_size))
        new_image.paste(resized_image, ((target_size - new_size[0]) // 2, (target_size - new_size[1]) // 2))

        return new_image

    def bytes_image_to_np(image_bytes, image_size=300):
        image = Image.open(io.BytesIO(image_bytes))
        image = pad_and_resize(image, image_size)
        image_array = np.array(image)
        image_array = image_array[:,:,[2,1,0]] / 255.0
        return image_array
    
    def first_frame_pad(input):
        padding_input = np.repeat(input[:1, ...], max_length - 1, axis=0)
        padded_input = np.concatenate([padding_input, input], axis=0)
        return padded_input
    
    while True:
        file_list_sampled = random.sample(file_list, file_batch_size)
        num_per_file = batch_size // file_batch_size
        natural_language_embedding = []
        image_high = []
        image_left = []
        image_right = []
        arms_action = []
        
        is_first = np.zeros((batch_size, max_length), dtype=bool)
                
        terminate_episode = np.zeros((batch_size, max_length, 3), dtype=np.int32)
        terminate_episode[:,:,1] = 1
        
        for i, file in enumerate(file_list_sampled):
            with h5py.File(file, 'r') as f:
                text_embedding = np.array(embedding_dict[f['instruction'][()].decode('utf-8')])
                length = f['action'].shape[0]
                assert length >= max_length, 'Trajectory length is shorter than max_length'
                action = np.array(f['action'])
                action = first_frame_pad(action)
                high = np.array(list(map(bytes_image_to_np, f['observations']['images']['cam_high'])))
                left = np.array(list(map(bytes_image_to_np, f['observations']['images']['cam_left_wrist'])))
                right = np.array(list(map(bytes_image_to_np, f['observations']['images']['cam_right_wrist'])))
                high = first_frame_pad(high)
                left = first_frame_pad(left)
                right = first_frame_pad(right)
                start = [np.random.randint(0, length - max_length + 1) for _ in range(num_per_file)]
                for j in range(num_per_file):
                    natural_language_embedding.append([text_embedding for _ in range(max_length)])
                    traj_l = f['action'].shape[0]
                    image_high.append(high[start[j]:start[j]+max_length])
                    image_left.append(left[start[j]:start[j]+max_length])
                    image_right.append(right[start[j]:start[j]+max_length])
                    arms_action.append(action[start[j]:start[j]+max_length])
        
        batch = {
            'is_first': np.array(is_first),
            'observation': {
                'image_high': np.array(image_high),
                'image_left': np.array(image_left),
                'image_right': np.array(image_right),
                'natural_language_embedding': np.array(natural_language_embedding)
                },
            'action': {
                'arms_l0': np.array(arms_action)[:,:,[0]],
                'arms_l1': np.array(arms_action)[:,:,[1]],
                'arms_l2': np.array(arms_action)[:,:,[2]],
                'arms_l3': np.array(arms_action)[:,:,[3]],
                'arms_l4': np.array(arms_action)[:,:,[4]],
                'arms_l5': np.array(arms_action)[:,:,[5]],
                'arms_l6': np.array(arms_action)[:,:,[6]],
                'arms_r0': np.array(arms_action)[:,:,[7]],
                'arms_r1': np.array(arms_action)[:,:,[8]],
                'arms_r2': np.array(arms_action)[:,:,[9]],
                'arms_r3': np.array(arms_action)[:,:,[10]],
                'arms_r4': np.array(arms_action)[:,:,[11]],
                'arms_r5': np.array(arms_action)[:,:,[12]],
                'arms_r6': np.array(arms_action)[:,:,[13]],
                'terminate_episode': terminate_episode
            },
            'is_last': np.zeros((batch_size, max_length), dtype=bool),
            'is_terminal': np.zeros((batch_size, max_length), dtype=bool)
            }
        
        return batch
        
file_list = [f'data/put_orange_paperbox/episode_{i}.hdf5' for i in range(20)]
file_batch_size = 8
batch_size = 16
embedding_dict = json.load(open('text_embeddings.json', 'r'))

batch = load_data_from_hdf5_2(file_list, batch_size, file_batch_size, embedding_dict)
pytree_display(batch)

2024-05-31 04:08:49.128273: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-31 04:08:49.128320: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-31 04:08:49.129594: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-05-31 04:08:55.700132: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
/data/zengguanning/miniconda3/envs/rtx/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebo

{
    "action": {
        "arms_l0": "Shape: (16, 32, 1)",
        "arms_l1": "Shape: (16, 32, 1)",
        "arms_l2": "Shape: (16, 32, 1)",
        "arms_l3": "Shape: (16, 32, 1)",
        "arms_l4": "Shape: (16, 32, 1)",
        "arms_l5": "Shape: (16, 32, 1)",
        "arms_l6": "Shape: (16, 32, 1)",
        "arms_r0": "Shape: (16, 32, 1)",
        "arms_r1": "Shape: (16, 32, 1)",
        "arms_r2": "Shape: (16, 32, 1)",
        "arms_r3": "Shape: (16, 32, 1)",
        "arms_r4": "Shape: (16, 32, 1)",
        "arms_r5": "Shape: (16, 32, 1)",
        "arms_r6": "Shape: (16, 32, 1)",
        "terminate_episode": "Shape: (16, 32, 3)"
    },
    "is_first": "Shape: (16, 32)",
    "is_last": "Shape: (16, 32)",
    "is_terminal": "Shape: (16, 32)",
    "observation": {
        "image_high": "Shape: (16, 32, 300, 300, 3)",
        "image_left": "Shape: (16, 32, 300, 300, 3)",
        "image_right": "Shape: (16, 32, 300, 300, 3)",
        "natural_language_embedding": "Shape: (16, 32, 512)"